In [80]:
import networkx as nx
import numpy as np

In [76]:
# asd


nc = np.load('./output/THREE_LOSSES_229_NC_ALL/fold4_cbt.npy')
adhd = np.load('./output/ADHD_THREE_LOSS_TRAIN/fold4_cbt.npy')
lmci = np.load('./output/LMCI_THREE_LOSS_TRAIN/fold4_cbt.npy')

In [92]:
#asd
asd = np.load('./output/THREE_LOSSES_229/fold0_cbt.npy')
asd_test = np.load('./output/test_data/fold0_all_cbts_asd.npy')

In [94]:
#nc
asd = np.load('./output/THREE_LOSSES_229_NC_ALL/fold0_cbt.npy')
asd_test = np.load('./output/test_data/fold0_all_cbts_nc.npy')

In [96]:
#adhd
asd = np.load('./output/ADHD_THREE_LOSS_TRAIN/fold0_cbt.npy')
asd_test = np.load('./output/ADHD_THREE_LOSS_TEST/fold0_all_cbts.npy')

In [98]:
#lmci
asd = np.load('./output/LMCI_THREE_LOSS_TRAIN/fold3_cbt.npy')
asd_test = np.load('./output/LMCI_THREE_LOSS_TEST/fold0_all_cbts.npy')

In [99]:
from scipy.sparse.csgraph import floyd_warshall

# Compute the shortest paths between all pairs of nodes
# Using floyd_warshall from scipy, which automatically considers 'asd' as a weighted graph
# We assume 'asd' is already defined and represents the adjacency matrix of the network
dist_matrix = floyd_warshall(asd, directed=False)

# Replace infinity values with zeros to avoid division by zero in later calculations
# Infinity values occur for pairs of nodes between which no path exists
dist_matrix[dist_matrix == np.inf] = 0

# Calculate the information centrality for each node
n_nodes = asd.shape[0]
info_centrality = np.zeros(n_nodes)

for i in range(n_nodes):
    # Exclude self-loops
    valid_distances = dist_matrix[i][dist_matrix[i] > 0]
    
    # Harmonic mean of distances for node i
    if valid_distances.size > 0:
        info_centrality[i] = 1 / np.mean(1 / valid_distances)

info_centrality.mean()

0.07724070826578083

In [89]:
total_degree_sum = np.sum(asd)
n=35

# Corrected Laplacian centrality calculation, focusing on non-negative contributions
C_Laplacian_corrected = np.zeros(n)

for i in range(n):
    # Directly using degree might already ensure non-negativity but let's ensure the calculation is clearly non-negative
    C_Laplacian_corrected[i] = np.sum(asd[i]) / total_degree_sum

C_Laplacian_corrected.mean()

0.02857142947614193

In [49]:
asd = (asd + asd.T) / 2
np.fill_diagonal(asd, 0)

for i in range(asd_test.shape[2]):
    asd_test[:, :, i] = (asd_test[:, :, i] + asd_test[:, :, i].T) / 2
    np.fill_diagonal(asd_test[:, :, i], 0)

# Calculate node strength for the main matrix
asd_strength = np.sum(asd, axis=1)

print("node strength", np.mean(asd_strength))

# Initialize an array to store the mean and std of strength differences for each test matrix
mean_strength_diffs = np.zeros(asd_test.shape[2])
std_strength_diffs = np.zeros(asd_test.shape[2])

# Calculate node strength for each test matrix and their differences from the main matrix
for i in range(asd_test.shape[2]):
    # Calculate node strength for the current test matrix
    test_matrix_strength = np.sum(asd_test[:, :, i], axis=1)
    
    # Calculate element-wise difference in node strength between the current test matrix and the main matrix
    strength_diffs = np.abs(test_matrix_strength - asd_strength)
    
    # Calculate the mean and std of the strength differences
    mean_strength_diffs[i] = np.mean(strength_diffs)
    std_strength_diffs[i] = np.std(strength_diffs)

np.mean(mean_strength_diffs), np.std(std_strength_diffs)

node strength 3.3048754


(0.9608889316208661, 0.09913314378828565)

In [24]:
np.mean([0.10420735836853082, 0.03982993397295201, 0.24534111144493254, 0.28904792516050254, 0.12435319609585263])

0.1605559050085541

In [ ]:
(0.10420735836853082, 0.23696557820263336)
(0.03982993397295201, 0.09707140581147687)
(0.24534111144493254, 1.0640493673628049)
(0.28904792516050254, 0.4822504313824283)
(0.12435319609585263, 0.1741636406797108)


In [64]:
def calculate_eigenvector_centrality(matrix, max_iter=100, tol=1.0e-6):
    n = matrix.shape[0]
    # Initialize the eigenvector with equal values (e.g., 1/n)
    x = np.ones(n) / n
    
    # Normalize the matrix by its largest absolute eigenvalue to ensure convergence
    eigenvalues, _ = np.linalg.eig(matrix)
    lambda_max = np.max(np.abs(eigenvalues))
    A_norm = matrix / lambda_max
    
    # Power iteration: repeatedly update the eigenvector estimate
    for _ in range(max_iter):
        x_next = A_norm.dot(x)
        x_next /= np.linalg.norm(x_next)  # Normalize the vector
        
        # Check for convergence
        if np.linalg.norm(x_next - x) < tol:
            break
        x = x_next
    
    return x

# Example usage with a dummy adjacency matrix
# Replace `asd` with your actual matrix
asd = np.random.rand(35, 35)

# Calculate eigenvector centrality
centrality = calculate_eigenvector_centrality(asd)
print(np.mean(centrality))

0.16804128110092748


In [76]:
def adjacency_matrix_to_list(matrix):
    adjacency_list = {}
    for i, row in enumerate(matrix):
        adjacency_list[i] = [j for j, connected in enumerate(row) if connected]
    return adjacency_list

def brandes_algorithm(graph):
    # Initialize betweenness centrality dictionary
    C_b = {v: 0 for v in graph}
    for s in graph:
        # Initialization
        S = []
        P = {v: [] for v in graph}  # Predecessors
        g = {v: 0 for v in graph}   # Number of shortest paths
        g[s] = 1
        d = {v: -1 for v in graph}  # Distance from s
        d[s] = 0
        Q = []  # Queue for BFS
        Q.append(s)
        
        # BFS from s
        while Q:
            v = Q.pop(0)
            S.append(v)
            for w in graph[v]:
                # Path discovery
                if d[w] < 0:
                    Q.append(w)
                    d[w] = d[v] + 1
                # Path counting
                if d[w] == d[v] + 1:
                    g[w] += g[v]
                    P[w].append(v)
        
        # Accumulation
        delta = {v: 0 for v in graph}
        while S:
            w = S.pop()
            for v in P[w]:
                delta[v] += (g[v] / g[w]) * (1 + delta[w])
            if w != s:
                C_b[w] += delta[w]
                
    return C_b

graph = adjacency_matrix_to_list(asd)

# Calculate betweenness centrality
betweenness_centrality = brandes_algorithm(graph)
print(np.mean(list(betweenness_centrality.values())))

0.05714285714285714


In [47]:
def calculate_betweenness_centrality(matrix):
    G = nx.from_numpy_array(matrix)
    centrality = nx.betweenness_centrality(G)
    return centrality

# Calculate centrality for the trained matrix
trained_centrality = calculate_betweenness_centrality(asd)
print("trained_centrality", np.mean(list(trained_centrality.values())))

# Calculate centralities for each testing set matrix and compute differences
differences = []
for i in range(asd_test.shape[2]):
    test_centrality = calculate_betweenness_centrality(asd_test[:, :, i])
    # Compute difference in centrality values
    # For betweenness, we're also comparing mean values as an example
    diff = np.abs(np.mean(list(test_centrality.values())) - np.mean(list(trained_centrality.values())))
    differences.append(diff)

# Calculate overall mean and standard deviation of differences
mean_difference = np.mean(differences)
std_difference = np.std(differences)

print("Mean Difference in Betweenness Centrality:", mean_difference)
print("Standard Deviation in Betweenness Centrality:", std_difference)

trained_centrality 5.092946269416858e-05
Mean Difference in Betweenness Centrality: 1.0913456291607553e-05
Standard Deviation in Betweenness Centrality: 2.089767778579806e-05
